In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import os, subprocess
import yt
import unyt
import warnings
import const
from scipy.integrate import quad


In [102]:
X, Y, Z = 0, 1, 2
ALPHA_EPS0P01, ALPHA_EPS0P1, ALPHA_EPS1P0, DMO, GAS = 0, 1, 2, 3, 4
HYDRO, DM, STAR = 0, 1, 2
epsilon = 1e-30

sim_idx = GAS


In [103]:
analysis_dir = "home/za9132/analysis"
sim_base_dir = "/home/za9132/scratch/romain"
sim_name = ["alpha_eps0p01", "alpha_eps0p1", "alpha_eps1p0", "dmo", "gas"][sim_idx]
sim_title = [r"$\varepsilon_{\rm SF} = 0.01$", r"$\varepsilon_{\rm SF} = 0.1$", r"$\varepsilon_{\rm SF} = 1.0$", "Dark Matter Only", "Multi-Freefall Model"][sim_idx]
sim_dir = os.path.join(sim_base_dir, sim_name)

os.chdir(sim_dir)


In [104]:
def get_stdout(cmd):
    ''' Return the standard output of a command line directive '''
    stdout = subprocess.check_output(cmd, shell=True).decode()
    return stdout


In [105]:
class Halo(object):
    
    def __init__(self, stdout_split):
        
        self.index = int(stdout_split[0])
        self.ncell = int(stdout_split[1])
        self.xyz = np.array(stdout_split[2:5], dtype=float)
        self.rho_max = float(stdout_split[5])
        self.mass = float(stdout_split[6])
        
class Clump(object):
    
    def __init__(self, stdout_split):
        
        stdout_split = stdout.split()
        
        self.index = int(stdout_split[0])
        self.level = int(stdout_split[1])
        self.parent_index = int(stdout_split[2])
        self.ncell = int(stdout_split[3])
        self.xyz = np.array(stdout_split[4:7], dtype=float)
        self.rho_border = float(stdout_split[7])
        self.rho_max = float(stdout_split[8])
        self.mass = float(stdout_split[9])
        

In [106]:
def a_exp_to_proper_time(a):
    '''Convert expansion factor to proper time.'''
    integrand = lambda a: (Omega_m0 * a**(-1) + Omega_k0 + Omega_L0 * a**2)**(-1/2)
    t = quad(integrand, 0, a)[0] / H0
    
    return t

def a_exp_to_conformal_time(a):
    '''Convert expansion factor to conformal time.'''
    integrand = lambda a: (Omega_m0 * a + Omega_k0 * a**2 + Omega_L0 * a**4)**(-1/2)
    tau = const.c * quad(integrand, 0, a)[0] / H0
    
    return tau

def proper_time_to_a_exp(t):
    '''Convert proper time to expansion rate.'''
    a = fsolve(lambda a: (a_exp_to_proper_time(a) - t) * H0, a_exp)
    
    return a

def conformal_time_to_a_exp(tau):
    '''Convert conformal time to expansion rate.'''
    a = fsolve(lambda a: (a_exp_to_conformal_time(a) - tau) * H0, a_exp)
    
    return a


In [107]:
def get_biggest_obj(num, find_halos=True, use_mass=True):
    
    list_of_obj_density = []
    list_of_obj_mass = []
    
    if find_halos:
        
        list_of_stdout_density = np.array(get_stdout("cat output*/halo* | sort -r -nk 2 | head -n %d" % num).split()).reshape(-1, 7)
        list_of_stdout_mass = np.array(get_stdout("cat output*/halo* | sort -r -gk 7 | head -n %d" % num).split()).reshape(-1, 7)
    
        for i in range(num):
            list_of_obj_density.append(Halo(list_of_stdout_density[i]))
            list_of_obj_mass.append(Halo(list_of_stdout_mass[i]))
    
    else:
        
        list_of_stdout_density = np.array(get_stdout("cat output*/clump* | sort -r -nk 2 | head -n %d" % num).split()).reshape(-1, 11)
        list_of_stdout_mass = np.array(get_stdout("cat output*/clump* | sort -r -gk 7 | head -n %d" % num).split()).reshape(-1, 11)
        
        for i in range(num):
            list_of_obj_density.append(Clump(list_of_stdout_density[i]))
            list_of_obj_mass.append(Clump(list_of_stdout_mass[i]))
        
    if list_of_obj_density[0].index != list_of_obj_mass[0].index:
        warnings.warn("Max density and max mass %s are not the same." % ["clumps", "halos"][find_halos])
    
    if use_mass:
        list_of_obj = list_of_obj_mass
    else:
        list_of_obj = list_of_obj_density
        
    return list_of_obj
    

In [108]:
halo = get_biggest_obj(1, find_halos=True, use_mass=True)[0]
info_file = get_stdout("ls output*/info*").split()[0]


In [109]:
ds = yt.load(info_file)


yt : [INFO     ] 2023-09-25 16:32:44,002 Parameters: current_time              = 4.006925024948247
yt : [INFO     ] 2023-09-25 16:32:44,003 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2023-09-25 16:32:44,004 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-09-25 16:32:44,004 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2023-09-25 16:32:44,005 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-09-25 16:32:44,005 Parameters: current_redshift          = 8.999851699314442
yt : [INFO     ] 2023-09-25 16:32:44,005 Parameters: omega_lambda              = 0.723999977111816
yt : [INFO     ] 2023-09-25 16:32:44,006 Parameters: omega_matter              = 0.275999993085861
yt : [INFO     ] 2023-09-25 16:32:44,006 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2023-09-25 16:32:44,006 Parameters: hubble_constant           = 0.703000030517578


In [117]:
a_exp = ds["aexp"]
redshift = 1 / a_exp - 1

H0 = ds["H0"] * const.km / const.Mpc
Omega_m0 = ds["omega_m"]
Omega_L0 = ds["omega_l"]
Omega_k0 = ds["omega_k"]
Omega_b0 = ds["omega_b"]
H = H0 * np.sqrt(Omega_m0 / a_exp**3 + Omega_k0 / a_exp**2 + Omega_L0)
rho_crit = 3 * H**2 / (8 * np.pi * const.G)
current_time = a_exp_to_proper_time(a_exp)

length_unit = ds["unit_l"]
density_unit = ds["unit_d"]
time_unit = ds["unit_t"]
mass_unit = density_unit * length_unit**3
velocity_unit = length_unit / time_unit
energy_unit = mass_unit * velocity_unit**2
energy_density_unit = density_unit * velocity_unit**2

amr_level = 13
# box_size = 10 * const.kpc / length_unit
box_size = 2 * const.kpc / length_unit
# left_edge = halo.xyz - box_size/2
left_edge = np.array([5.105151466E-01, 5.127534424E-01, 4.929928780E-01]) - box_size / 2
N = int(box_size * ds.domain_dimensions[0] * 2**amr_level)

dx = box_size / N * length_unit
dV = dx**3


In [118]:
data = ds.covering_grid(level=amr_level, left_edge=left_edge, dims=[N]*3)


In [119]:
particle_type = data["io", "particle_family"].value.astype(int)
is_dm = particle_type == DM
is_star = particle_type == STAR

dm_coord = (np.array([data["io", "particle_position_x"].value[is_dm], data["io", "particle_position_y"].value[is_dm], data["io", "particle_position_z"].value[is_dm]]) - left_edge[:, None] - box_size / 2) * length_unit
dm_mass = data["io", "particle_mass"][is_dm] * mass_unit

universe_age = a_exp_to_proper_time(1.)
star_coord = (np.array([data["io", "particle_position_x"].value[is_star], data["io", "particle_position_y"].value[is_star], data["io", "particle_position_z"].value[is_star]]) - left_edge[:, None] - box_size / 2) * length_unit
star_mass = data["io", "particle_mass"][is_star] * mass_unit
star_birth_time = (data["io", "conformal_birth_time"][is_star].value / H0 + universe_age)


yt : [WARNING  ] 2023-09-25 16:35:25,354 The DM particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-25 16:35:25,355 The star particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-25 16:35:25,355 The cloud particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-25 16:35:25,355 The dust particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-25 16:35:25,356 The star_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-25 16:35:25,356 The cloud_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-25 16:35:25,356 The dust_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-25 16:35:25,356 The gas_tracer particle filter already exists. Overriding.
yt : [INFO     ] 2023-09-25 16:35:25,357 Adding particle_type: DM
yt : [INFO     ] 2023-09-25 16:35:25,366 Adding particle_type: star
yt : [INFO     ] 2023-09-25 16:35:25,375 Adding particle_type: cloud
yt : [INFO     ] 2

In [120]:
coord = (np.array([data["index", "x"].value, data["index", "y"].value, data["index", "z"].value]) - left_edge[:, None, None, None] - box_size / 2) * length_unit
density = data["ramses", "Density"].value * density_unit
metallicity = data["ramses", "Metallicity"].value 
pressure = data["ramses", "Pressure"].value * energy_density_unit
turb_energy = data["ramses", "hydro_scalar_01"].value * velocity_unit**2
refinement_criterion = data["ramses", "hydro_scalar_02"].value


In [121]:
r = np.sqrt(np.sum(coord**2, axis=0))
r_dm = np.sqrt(np.sum(dm_coord**2, axis=0))
r_star = np.sqrt(np.sum(star_coord**2, axis=0))
temperature = pressure / density * const.m_H / const.k_B


In [122]:
np.savez(
    file="gridded_data_romain",
    halo_idx=halo.index,
    halo_mass=halo.mass*mass_unit,
    a_exp=a_exp,
    redshift=redshift,
    current_time=current_time,
    H0=H0,
    Omega_m0=Omega_m0,
    Omega_L0=Omega_L0,
    Omega_k0=Omega_k0,
    Omega_b0=Omega_b0,
    H=H,
    rho_crit=rho_crit,
    length_unit=length_unit,
    density_unit=density_unit,
    time_unit=time_unit,
    mass_unit=mass_unit,
    velocity_unit=velocity_unit,
    energy_unit=energy_unit,
    energy_density_unit=energy_density_unit,
    amr_level=13,
    box_size=box_size,
    left_edge=left_edge,
    N=N,
    dx=dx,
    dV=dV,
    dm_coord=dm_coord,
    dm_mass=dm_mass,
    star_coord=star_coord,
    star_mass=star_mass,
    star_birth_time=star_birth_time,
    coord=coord,
    density=density,
    metallicity=metallicity,
    pressure=pressure,
    turb_energy=turb_energy,
    refinement_criterion=refinement_criterion,
    r=r,
    r_dm=r_dm,
    r_star=r_star,
    temperature=temperature
)
